In [15]:
import pandas as pd
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [10]:
train_raw_data = pd.read_csv("train_raw_data.csv", sep = '\t')
test_raw_data = pd.read_csv("test_raw_data.csv", sep = '\t') 
y_train = pd.read_csv("y_train.csv", sep = '\t' )
y_test = pd.read_csv("y_test.csv", sep = '\t' )
features_for_train = pd.read_csv("features_for_train.csv", sep = '\t')
features_for_test = pd.read_csv("features_for_test.csv", sep = ',')
features_for_test.drop('A', axis = 1, inplace = True)#has another format
features_for_test['9'] = features_for_test['9'].map({False : 0.0, True : 1.0})
features_for_train['9'] = features_for_train['9'].map({False : 0.0, True : 1.0})

In [11]:
def preprocess_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    stemmer = None #PorterStemmer()
    tokens = word_tokenize(sentence)
    lemmas = []
    skip = False
    for i in range(len(tokens)):
        if tokens[i] == '@':
            skip = True
        elif skip:
            skip = False
        else:
            lemmas.append(lemmatizer.lemmatize(tokens[i])) 
    stems = []
    if stemmer != None:
        for lemma in lemmas:
            stems.append(stemmer.stem(lemma))
        return " ".join(stems)
    return " ".join(lemmas)

train_raw_data['content'] = list(map(preprocess_sentence, train_raw_data['content']))
test_raw_data['content'] = list(map(preprocess_sentence, test_raw_data['content']))
vectorizer = TfidfVectorizer(stop_words = 'english')
X_train_TF = pd.DataFrame(vectorizer.fit_transform(train_raw_data['content']).todense())
X_test_TF = pd.DataFrame(vectorizer.transform(test_raw_data['content']).todense())
X_train = pd.merge(X_train_TF, features_for_train, how='outer', left_index=True, right_index=True)
X_test = pd.merge(X_test_TF, features_for_test, how='outer', left_index=True, right_index=True)

In [19]:
random_forest = RandomForestClassifier(n_estimators = 25, criterion = 'gini')
random_forest.fit(X_train.as_matrix(), y_train.as_matrix())
print(random_forest.score(X_test.as_matrix(), y_test.as_matrix()))

/home/george/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


0.438448028102
